# Audio Recommender
### Pin-Hao Chen (phc2121) | Sharon Tsao (sjt2141)

In [1]:
from pyspark.mllib.recommendation import *
from operator import *
from pyspark.sql.types import *

In [2]:
rawUserArtistData = sc.textFile("profiledata_06-May-2005/user_artist_data.txt")
rawArtistData = sc.textFile("profiledata_06-May-2005/artist_data.txt")
rawArtistAlias = sc.textFile("profiledata_06-May-2005/artist_alias.txt")

In [3]:
def helper1(line):
    pairs = line.split("\t")
    return (pairs[0], pairs[1].strip()) if len(pairs) == 2 else (None, None)

artistByID = rawArtistData.flatMap(lambda l: (helper1(l), ))
artist_df = artistByID.toDF(['artistid', 'name'])
artist_df = artist_df.filter(artist_df.artistid != "null")
# artist_df.show(5)

In [4]:
def helper2(line):
    tokens = line.split("\t")  
    return (tokens[0], tokens[1].strip()) if tokens[0] else (None, None)   
    
bad_good = rawArtistAlias.flatMap(lambda l: (helper2(l), ))
artistAlias = bad_good.collectAsMap()
artistAlias.pop(None, None)
# artist_alias_df = bad_good.toDF(['badid', 'goodid'])
# artist_alias_df = artist_alias_df.filter(artist_alias_df.badid != "null")
# artist_alias_df.show(5)

In [5]:
def helper3(line):
    userID, artistID, count = line.split(" ")
    finalArtistID = artistID
    if artistID in bArtistAlias.value:
        finalArtistID = bArtistAlias.value[artistID]
    return Rating(int(userID), int(finalArtistID), int(count))

bArtistAlias = sc.broadcast(artistAlias)
trainData = rawUserArtistData.map(lambda l: helper3(l)).cache()
#userArtist_df = trainData.toDF(["userid", "artistid", "playcount"])

In [6]:
model = ALS.trainImplicit(trainData, 10, iterations=5, lambda_=0.01, blocks=1)

In [7]:
target_user = 2093760
recommendations = model.recommendProducts(target_user, 10)

In [8]:
recommendations

[Rating(user=2093760, product=1007614, rating=0.03586678662102625),
 Rating(user=2093760, product=4605, rating=0.03431157685151171),
 Rating(user=2093760, product=2814, rating=0.03352224554775464),
 Rating(user=2093760, product=1037970, rating=0.032523810056845674),
 Rating(user=2093760, product=829, rating=0.03213284150739629),
 Rating(user=2093760, product=1001819, rating=0.032074268863427624),
 Rating(user=2093760, product=1811, rating=0.03041561913936728),
 Rating(user=2093760, product=250, rating=0.02903475223594766),
 Rating(user=2093760, product=930, rating=0.02745748509540613),
 Rating(user=2093760, product=1003249, rating=0.02676261953314412)]

In [9]:
recommendedProductIDs = set(map(lambda r: r.product, recommendations))
recommendedProductIDs

{250, 829, 930, 1811, 2814, 4605, 1001819, 1003249, 1007614, 1037970}

In [10]:
rmd_lt = set(map(lambda e: e.name, artist_df.filter(artist_df.artistid.isin(recommendedProductIDs)).collect()))
rmd_lt

{'2Pac',
 '50 Cent',
 'Dr. Dre',
 'Eminem',
 'Jay-Z',
 'Kanye West',
 'Ludacris',
 'Nas',
 'Outkast',
 'Snoop Dogg'}